In [2]:
from pyspark.sql import SparkSession
import os

SOURCE_CSV_FILE = os.path.abspath("./datasets/rangalamahesh_bank_churn.csv")
# Khởi tạo Spark session
spark = SparkSession.builder.appName("ChurnDataSQL").getOrCreate()

# Đọc dữ liệu từ tệp CSV
df = spark.read.csv(SOURCE_CSV_FILE, header=True, inferSchema=True)

# Hiển thị schema của dữ liệu
df.printSchema()

# Tạo bảng tạm thời cho dữ liệu để thực hiện các truy vấn SQL
df.createOrReplaceTempView("churn_data")

print("1. Tính toán tỷ lệ giữ lại khách hàng theo quốc gia và giới tính")

retention_rate_by_country_gender = spark.sql("""
    SELECT Geography, Gender, 
           SUM(CASE WHEN IsActiveMember = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) AS RetentionRate
    FROM churn_data
    GROUP BY Geography, Gender
    ORDER BY RetentionRate DESC
""")
retention_rate_by_country_gender.show()

print("2. Tìm 5 khách hàng có số dư lớn nhất trong mỗi quốc gia")
top_balance_by_country = spark.sql("""
    SELECT CustomerId, Geography, Balance
    FROM (
        SELECT CustomerId, Geography, Balance,
               ROW_NUMBER() OVER (PARTITION BY Geography ORDER BY Balance DESC) AS rank
        FROM churn_data
    ) ranked
    WHERE rank <= 5
    ORDER BY Geography, rank
""")
top_balance_by_country.show()

print("3. Tính toán số dư trung bình và tổng số sản phẩm cho mỗi nhóm tuổi (Age Group)")
balance_and_products_by_age_group = spark.sql("""
    SELECT AgeGroup, 
           AVG(Balance) AS AvgBalance, 
           SUM(NumOfProducts) AS TotalProducts
    FROM (
        SELECT *,
               CASE 
                   WHEN Age < 25 THEN 'Under 25'
                   WHEN Age BETWEEN 25 AND 34 THEN '25-34'
                   WHEN Age BETWEEN 35 AND 44 THEN '35-44'
                   WHEN Age BETWEEN 45 AND 54 THEN '45-54'
                   ELSE '55+' 
               END AS AgeGroup
        FROM churn_data
    ) grouped
    GROUP BY AgeGroup
    ORDER BY AgeGroup
""")
balance_and_products_by_age_group.show()

print("4. Tính toán tổng số dư và số dư trung bình của các khách hàng đã hoạt động trong hơn 5 năm và có thẻ tín dụng")
active_customers_balance = spark.sql("""
    SELECT SUM(Balance) AS TotalBalance, 
           AVG(Balance) AS AvgBalance
    FROM churn_data
    WHERE Tenure > 5 
      AND HasCrCard = 1
""")
active_customers_balance.show()

print("5. Tìm các khách hàng có mức lương ước tính bất thường so với số dư tài khoản bằng cách sử dụng z-score")
from pyspark.sql.functions import mean, stddev

# Tính toán mean và stddev của Balance và EstimatedSalary
stats = df.select(mean("Balance").alias("mean_balance"),
                  stddev("Balance").alias("std_balance"),
                  mean("EstimatedSalary").alias("mean_salary"),
                  stddev("EstimatedSalary").alias("std_salary")).collect()

mean_balance = stats[0]["mean_balance"]
std_balance = stats[0]["std_balance"]
mean_salary = stats[0]["mean_salary"]
std_salary = stats[0]["std_salary"]

# Thêm cột z-score cho Balance và EstimatedSalary
df = df.withColumn("z_score_balance", (df.Balance - mean_balance) / std_balance)
df = df.withColumn("z_score_salary", (df.EstimatedSalary - mean_salary) / std_salary)

# Tạo bảng tạm thời mới cho dữ liệu đã được thêm z-score
df.createOrReplaceTempView("churn_data_with_z")

# Truy vấn các khách hàng có z-score của EstimatedSalary cao hơn ngưỡng nhất định (ví dụ: 2) so với z-score của Balance
anomalous_customers = spark.sql("""
    SELECT CustomerId, Balance, EstimatedSalary, z_score_balance, z_score_salary
    FROM churn_data_with_z
    WHERE ABS(z_score_salary - z_score_balance) > 2
    ORDER BY ABS(z_score_salary - z_score_balance) DESC
""")
anomalous_customers.show()



root
 |-- id: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: double (nullable = true)
 |-- IsActiveMember: double (nullable = true)
 |-- EstimatedSalary: double (nullable = true)

1. Tính toán tỷ lệ giữ lại khách hàng theo quốc gia và giới tính
+---------+------+-----------------+
|Geography|Gender|    RetentionRate|
+---------+------+-----------------+
|   France|  Male|51.77984590317859|
|    Spain|  Male|51.55393053016453|
|    Spain|Female|49.15999218597382|
|   France|Female|48.64985163204748|
|  Germany|  Male|48.21665560716656|
|  Germany|Female|43.41633740696499|
+---------+------+-----------------+

2. Tìm 5 khách hàng có số